In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import math
from gkpd.tensorops import kron
import time 

In [2]:
cal_time = []
for i in enumerate(range(100)):
    a = torch.randn(10000,10000)
    b = torch.randn(10000,10000)
    a = a.cuda()
    b = b.cuda()
    start = time.time()
    c = torch.matmul(a,b)
    end = time.time()
    cal_time.append(end-start)
cal_time = np.array(cal_time)
print(np.mean(cal_time), np.std(cal_time))

0.0003479146957397461 0.0019947799501378533


In [3]:
# keep the same size, but a have a sparsity of 0.1
sparsity = 0.1
a = torch.randn(10000,10000)
a_shape = a.flatten().shape[0]
sparse_time = []
mask = torch.cat([torch.zeros(int(a_shape*sparsity)), torch.ones(int(a_shape*(1-sparsity)))])
mask = mask[torch.randperm(a_shape)]
mask = mask.reshape(a.shape)
for i in enumerate(range(100)):
    a = torch.randn(10000,10000)
    b = torch.randn(10000,10000)
    a = a * mask
    a = a.cuda()
    b = b.cuda()
    start = time.time()
    c = torch.matmul(a,b)
    end = time.time()
    sparse_time.append(end-start)
sparse_time = np.array(sparse_time)
print(np.mean(sparse_time), np.std(sparse_time))

0.00013594865798950196 1.608695843083442e-05


In [ ]:
a = torch.randn(100, 100)


In [4]:
import torch 
torch.__version__

'2.1.1+cu121'

In [1]:
import torch
import numpy as np
def quick_kron(tensora, tensorb):
    """a quick kronecker product for two tensors

    For example, Tensor A of size [x1, y1] kronecker product with Tensor B of size [x2, y2], will get a tensor of size [x1*x2, y1*y2]. This can be extend to multiple dimension two tensors.     
    Args:
        tensora (_type_): Tensor Matrix A
        tensorb (_type_): Tensor Matrix B
    """
    a_shape = np.array(tensora.shape)
    b_shape = np.array(tensorb.shape)
    out_shape = np.multiply(a_shape, b_shape)
    return  torch.einsum('ik,jl', tensora, tensorb).reshape(*out_shape)
    
    
    
    

In [2]:
import time 
device = torch.device('cuda:1')
x1 = torch.randn(10, 10)
x2 = torch.randn(15, 10)
x1 = x1.to(device)
x2 = x2.to(device)
start1 = time.time()
x3 = torch.kron(x1, x2)
end1 = time.time()
print(end1-start1)
start2 = time.time()
x4 = quick_kron(x1, x2)
end2 = time.time()
print(end2-start2)
x3==x4

0.02580547332763672
0.009335994720458984


tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:1')